![](/files/images/dbxatscale.png)

# Accuracy UDF
Run this notebook using <code>%run ./05_Accuracy_UDF</code> to manually <b>call the accuracy checking function</b> for the m5_walmart_sales model that predicts item category. A message will be automatically sent to slack with the results.

In [0]:
# imports and session creation
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

import warnings
warnings.filterwarnings("ignore")

In [0]:
%run ./XX_Establish_AtScale_Connection

In [0]:
# Create function
def m5_accuracy():

  # import model
  import mlflow
  from datetime import datetime
  model_name = "Databricks_AI_Link_M5_Demo"
  model_version = 4
  model_uri=f"models:/{model_name}/{model_version}"
  model = mlflow.pyfunc.load_model(model_uri=model_uri)

  # get dataframe 
  df = spark.sql(query).toPandas()

  # make codified categories column
  item_cat=[]
  for index, row in df.iterrows():
    item = row["item"][0:3]
    if item == "FOO":
      item_cat.append(1)
    elif item == "HOB":
      item_cat.append(2)
    elif item == "HOU":
      item_cat.append(3)
  
  df["item_cat"] = item_cat

  # drop the item column
  df = df.drop(["item"], axis = 1)

  # make prediction column
  df["predicted_class"] = model.predict(df)

  # make grade column
  arr = []
  correct = 0
  for i in range(df.shape[0]):
    if(df["predicted_class"][i] == df["item_cat"][i]):
      arr.append(1)
      correct+=1
    else:
      arr.append(0)
  
  ### Uncomment to add a binary column to see item by item correctness
  # df["successful_predict"] = arr
  # display(df)

  return str(round(correct / df.shape[0], 4) * 100)

In [0]:
# Call 
success_rate = m5_accuracy()
print(success_rate)

2023/08/04 17:58:07 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module 'configparser' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.


[LightGBM] [Warning] lambda_l2 is set=0.176191006422464, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.176191006422464
[LightGBM] [Warning] lambda_l1 is set=0.2347409701792684, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2347409701792684
99.72


Please choose a project:
Automatically selecting only option: "ID: bfab4719-9a61-46f5-7d4c-1dcc2881ba44: Name: Walmart Databricks ML"
Please choose a published project:
Automatically selecting only option: "ID: 7f2140aa-8b8c-4517-6484-47432da7e337: Name: Walmart Databricks ML"
Please choose a data model:
Automatically selecting only option: "ID: 50fd61ba-2a0b-44d2-7dec-786e15c6ab6f: Name: m5_walmart_sales"


AtScale connection successful.


### Create Webhook 
This will let our developers know when their model has been retrained so they can monitor their compute cost and such.

In [0]:
%run ./06_Webhook_Set_Up

In [0]:
now = datetime.now()
success_message = "\n\nSuccess Rate: " + str(success_rate) + "%"

In [0]:
#format success message
messages = ["V--------------ACCURACY TEST----------------V"]
messages += ["DATA MODEL: " + data_model]
messages += ["DATE: " + str(now)]
messages += [success_message]
messages += ["*******************************************"]
messages += [" "]
messages += [" "]
messages += [" "] 

Webhook Set Up Complete.


In [0]:
send_slack_messages(slack_url, channel, messages)

![](/files/images/atscale_logo.png)